In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_columns = None
pd.set_option("display.max_colwidth", -1)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.max_rows", 101)

/home/nathaniel/anaconda3/envs/nba-predictor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [15]:
DATA_PATH = 'data'

SEASONS_PROCESSED_DS = f"{DATA_PATH}/seasons.processed.feather"

TEAMS_DS = f"{DATA_PATH}/teams.csv"
TEAMS_PROCESSED_DS = f"{DATA_PATH}/teams.processed.feather"

RANKING_DS = f"{DATA_PATH}/ranking.csv"
RANKING_PROCESSED_DS = f"{DATA_PATH}/ranking.processed.feather"

GAMES_DS = f"{DATA_PATH}/games.csv"
GAMES_PROCESSED_DS = f"{DATA_PATH}/games.processed.feather"

In [26]:
games = pd.read_csv(GAMES_DS, parse_dates=["GAME_DATE_EST"], usecols=["GAME_ID",'GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home',
       'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away',
       'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away',
       'HOME_TEAM_WINS']
                    , infer_datetime_format=True, index_col="GAME_ID")
games = games.sort_values(by=['GAME_DATE_EST', 'GAME_ID'])

teams = pd.read_feather(TEAMS_PROCESSED_DS)

seasons = pd.read_feather(SEASONS_PROCESSED_DS)

rankings = pd.read_csv(RANKING_DS, parse_dates=["STANDINGSDATE"], usecols=['TEAM_ID', 'LEAGUE_ID', 'SEASON_ID', 'STANDINGSDATE', 'CONFERENCE',
       'TEAM', 'G', 'W', 'L', 'W_PCT', 'HOME_RECORD', 'ROAD_RECORD'], infer_datetime_format=True,
                      index_col=["STANDINGSDATE"])

games_matchup = pd.read_feather(GAMES_PROCESSED_DS)
games_matchup = games_matchup.set_index(["GAME_ID"])
games_matchup = games_matchup.sort_values(by=['GAME_DATE_EST', 'GAME_ID'])

In [39]:
df = rankings

In [ ]:
df.index.value_counts()

In [ ]:
df_2015 = df[df.index.year == 2015]

In [ ]:
df_gws = df_2015[df_2015.TEAM_ID == 1610612744]

In [ ]:
df_gws.sort_index()
#rankings_g = rankings[rankings.SEASON_ID >= 1015].groupby(["STANDINGSDATE"])
#rankings_g.describe()

2018-07-17    30
2015-03-19    30
2007-10-10    30
2012-05-04    30
2016-11-27    30
              ..
2004-09-29    29
2004-01-05    29
2004-08-07    29
2004-03-23    29
2004-07-06    29
Name: STANDINGSDATE, Length: 5997, dtype: int64

In [58]:
df_2015 = df[df.index.year == 2015]

In [60]:
df_gws = df_2015[df_2015.TEAM_ID == 1610612744]

In [62]:
df_gws.sort_index()
#rankings_g = rankings[rankings.SEASON_ID >= 1015].groupby(["STANDINGSDATE"])
#rankings_g.describe()

,TEAM_ID,LEAGUE_ID,SEASON_ID,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
STANDINGSDATE,,,,,,,,,,,
2015-01-01,1610612744,0,22014,West,Golden State,30,25,5,0.833,12-1,13-4
2015-01-02,1610612744,0,22014,West,Golden State,31,26,5,0.839,13-1,13-4
2015-01-03,1610612744,0,22014,West,Golden State,31,26,5,0.839,13-1,13-4
2015-01-04,1610612744,0,22014,West,Golden State,31,26,5,0.839,13-1,13-4
2015-01-05,1610612744,0,22014,West,Golden State,32,27,5,0.844,14-1,13-4
...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,1610612744,0,22015,West,Golden State,29,28,1,0.966,14-0,14-1
2015-12-28,1610612744,0,22015,West,Golden State,30,29,1,0.967,15-0,14-1
2015-12-29,1610612744,0,22015,West,Golden State,30,29,1,0.967,15-0,14-1
